In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
# 读取数据并预处理
data_file = "data/game/network0.csv"  # 替换为你的数据文件路径
data = pd.read_csv(data_file)

# 提取时间列和边列
time_col = "TIME"
edge_cols = [col for col in data.columns if col != time_col]

# 获取所有节点名称（列名中的 P1, P2 等）
all_nodes = sorted(set(node.split("_TO_")[0] for node in edge_cols)|set(node.split("_TO_")[1] for node in edge_cols))
num_nodes = len(all_nodes)
node_index = {node: i for i, node in enumerate(all_nodes)}

# 初始化 T x N x N 矩阵
timesteps = data[time_col].unique()
T = len(timesteps)
N = num_nodes
adj_matrices = np.zeros((T, N, N), dtype=int)

# 构造邻接矩阵
for t_idx, timestep in enumerate(timesteps):
    # 获取当前时间戳的边数据
    snapshot = data[data[time_col] == timestep]
    
    for edge in edge_cols:
        value = snapshot[edge].values[0]  # 当前时间戳该边的值
        if value == 1:
            node1, node2 = edge.split("_TO_")
            i, j = node_index[node1], node_index[node2]
            adj_matrices[t_idx, i, j] = 1
            adj_matrices[t_idx, j, i] = 1  # 无向性

# 将稀疏矩阵转换为压缩行格式
sparse_matrices = [sp.csr_matrix(adj_matrix) for adj_matrix in adj_matrices]
# 假设 sparse_matrices 是一个包含多个 csr_matrix 的列表
adj_data = np.concatenate([mat.data for mat in sparse_matrices])
adj_indices = np.concatenate([mat.indices for mat in sparse_matrices])

# 正确合并 indptr
indptr_list = [mat.indptr for mat in sparse_matrices]
offsets = np.cumsum([0] + [len(mat.data) for mat in sparse_matrices[:-1]])
#adj_indptr = np.concatenate([indptr + offset for indptr, offset in zip(indptr_list, offsets)])
adj_indptr = np.concatenate([indptr[:-1] + offset for indptr, offset in zip(indptr_list, offsets)])
adj_indptr = np.append(adj_indptr, indptr_list[-1][-1] + offsets[-1])
# 假设所有矩阵形状相同
adj_shape = (sum(mat.shape[0] for mat in sparse_matrices), sparse_matrices[0].shape[1])

adj_dict = {
    'adj_data': adj_data,
    'adj_indices': adj_indices,
    'adj_indptr': adj_indptr,
    'adj_shape': adj_shape
}

# 保存字典
np.save('game_network_0.npy', adj_dict)
# 保存为字典格式
#np.save("game_network_0.npy", adj_dict)
# 输出结果
#print("Temporal Adjacency Matrix Shape:", adj_matrices.shape)

# 如果需要保存结果为文件，例如 numpy 的 .npy 文件
#np.save("game_network_0.npy", adj_matrices)


In [2]:
all_nodes

['LAPTOP', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']